## AWQ Mistral


In [3]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch
import os
from prompts import classifier_prompt

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [5]:
weights_dir = "./weights"
model_name_or_path = "TheBloke/Mistral-7B-v0.1-AWQ"

# Load model
model = AutoAWQForCausalLM.from_quantized(
    model_name_or_path,
    fuse_layers=True,
    trust_remote_code=False,
    safetensors=True,
    device=device,
    cache_dir=weights_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=False, device=device, cache_dir=weights_dir
)

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Fusing layers...: 100%|██████████| 32/32 [00:01<00:00, 19.29it/s]


### Test model


In [11]:
user_query = "Move robot tcp left for 1000mm"
prompt_template = f"""{classifier_prompt.format(user_query=user_query)}

"""

print("\n\n*** Generate:")

tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens, do_sample=True, temperature=0.1, top_p=0.95, top_k=40, max_new_tokens=30
)

print("Output: ", tokenizer.decode(generation_output[0]))



*** Generate:
Output:  <s> 
# CONTEXT
You will be presented with user query and your job is to classifie the function that will be executed based on user query. 
Function names and explanations:
- move_robot_tcp: move robot tool center point (TCP) to location.
- move_joint: rotate or move specific robot joint.
- get_joint_values: get information about robot joints (eg. angles, position etc.). 

Choose ONLY from the list of functions provided.
Your output MUST BE only function name.
STOP AFTER GIVING FUNCTION NAME

# USER QUERY:
Move robot tcp left for 1000mm


# OUTPUT:
move_robot_tcp


# USER QUERY:
Move robot joint 1 to 1


### Test pipeline


In [ ]:
# Inference should be possible with transformers pipeline as well in future
# But currently this is not yet supported by AutoAWQ (correct as of September 25th 2023)
from transformers import pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
    device=device,
)

print(pipe(prompt_template)[0]["generated_text"])

## GPTQ Mistral


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [9]:
weights_dir = "./weights"
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    trust_remote_code=False,
    cache_dir=weights_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=False, device=device, cache_dir=weights_dir
)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

### Test model


In [ ]:
prompt = "Tell me about AI"
prompt_template = f"""{prompt}

"""

print("\n\n*** Generate:")

tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens, do_sample=True, temperature=0.1, top_p=0.95, top_k=40, max_new_tokens=512
)

print("Output: ", tokenizer.decode(generation_output[0]))

## JSON former


In [ ]:
from jsonformer import Jsonformer

In [ ]:
json_schema = {
    "type": "object",
    "properties": {
        "function_name": {"type": "string"},
    },
}

user_query = "Move robot tcp left for 1000mm"
prompt_template = f"""{classifier_prompt.format(user_query=user_query)}"""
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt_template)
generated_data = jsonformer()

print(generated_data)

AttributeError: 'MistralAWQForCausalLM' object has no attribute 'device'